In [1]:
import pandas as pd

In [2]:
OI_train_file = 'oidv6-train-images-with-labels-with-rotation.parquet'
train_df = pd.read_parquet(OI_train_file)

print("OI_train_file columns:", train_df.columns)
print("OI_train_file shape:", train_df.shape)
train_df.head()

OI_train_file columns: Index(['ImageID', 'Subset', 'OriginalURL', 'OriginalLandingURL', 'License',
       'AuthorProfileURL', 'Author', 'Title', 'OriginalSize', 'OriginalMD5',
       'Thumbnail300KURL', 'Rotation'],
      dtype='object')
OI_train_file shape: (7337077, 12)


,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,4fa8054781a4c382,train,https://farm3.staticflickr.com/5310/5898076654...,https://www.flickr.com/photos/michael-beat/589...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/michael-beat/,Michael Beat,...die FNF-Kerze,4405052,KFukvivpCM5QXl5SqKe41g==,https://c1.staticflickr.com/6/5310/5898076654_...,0.0
1,d05c3e451f79174d,train,https://c6.staticflickr.com/8/7375/13791738583...,https://www.flickr.com/photos/jeepersmedia/137...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/jeepersmedia/,Mike Mozart,Pez Kids Breakfast Cereal Prizes,972558,KoVMrVWKftzadg1EGr6nCg==,https://c1.staticflickr.com/8/7375/13791738583...,0.0
2,5bde9ad21e82deed,train,https://c6.staticflickr.com/1/223/495658260_89...,https://www.flickr.com/photos/8106459@N07/4956...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/8106459@N07/,David~O,Isabella Lake,234199,dsfj7rNAbGq67OzPRAGdsQ==,https://c2.staticflickr.com/1/223/495658260_85...,0.0
3,b37f763ae67d0888,train,https://c1.staticflickr.com/1/67/197493648_628...,https://www.flickr.com/photos/drstarbuck/19749...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/drstarbuck/,Karen,Three boys on a hill,494555,9IzEn38GRNsVpATuv7gzEA==,https://c3.staticflickr.com/1/67/197493648_628...,0.0
4,3231e75bd7120b7a,train,https://c3.staticflickr.com/4/3654/3619297168_...,https://www.flickr.com/photos/cckaiser/3619297168,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/cckaiser/,Colin,DSC_0096,2570210,195AcCHMqin3MYTv+pTDuA==,https://c4.staticflickr.com/4/3654/3619297168_...,0.0


In [3]:
OI_label_file = 'oidv7-train-annotations-human-imagelabels.parquet'
label_df = pd.read_parquet(OI_label_file)

print("OI_label_file columns:", label_df.columns)
print("OI_label_file shape:", label_df.shape)
label_df.head()

OI_label_file columns: Index(['ImageID', 'Source', 'LabelName', 'Confidence'], dtype='object')
OI_label_file shape: (58783034, 4)


,ImageID,Source,LabelName,Confidence
0,000002b66c9c498e,verification,/m/010l12,0.0
1,000002b66c9c498e,verification,/m/0118n_9r,0.0
2,000002b66c9c498e,verification,/m/011_f4,0.0
3,000002b66c9c498e,verification,/m/011l78,0.0
4,000002b66c9c498e,verification,/m/011s0,0.0


In [4]:
class_df = pd.read_csv('oidv7-class-descriptions.csv')

print(class_df.shape)
class_df.head()

(20931, 2)


,LabelName,DisplayName
0,/m/0c4936,'Nduja
1,/m/06w6y06,10 cane
2,/m/079zcf,100 metres hurdles
3,/m/02pv3hz,100plus
4,/m/0bb154,110 metres hurdles


In [5]:
train_df['Rotation'].value_counts()

Rotation
0.0      6115548
270.0      95526
90.0       45440
180.0      10241
Name: count, dtype: int64

In [6]:
train_df = train_df[train_df['Rotation'] == 0.0]
print(train_df.shape)

(6115548, 12)


In [7]:
size_df = train_df['OriginalSize'].value_counts().sort_index(ascending=False, inplace=False)

size_df


OriginalSize
15727733    1
15727673    1
15727633    1
15727581    1
15727472    1
           ..
3446        1
2822        1
2555        1
2498        1
1804        1
Name: count, Length: 3434017, dtype: int64

In [8]:
# Lower bound: 10 KB (keeps reasonable photos)
# Upper bound: 30 MB (protects against weird corrupted multimegabyte files)
clean_train_df = train_df[(train_df['OriginalSize'] > 10_000) & (train_df['OriginalSize'] < 30_000_000)]

print(clean_train_df.shape)

(6115335, 12)


In [9]:
label_df['Source'].value_counts()

Source
verification                48965561
crowdsource-verification     9817473
Name: count, dtype: int64

In [10]:
label_df['Confidence'].value_counts()

Confidence
0.0    37638859
1.0    21144175
Name: count, dtype: int64

In [11]:
label_df = label_df[label_df['Confidence'] == 1.0]

label_df.shape

(21144175, 4)

In [12]:
inhouse_label_df = label_df[label_df['Source'] == 'verification']

inhouse_label_df.shape

(14703564, 4)

In [13]:
outsourced_label_df = label_df[label_df['Source'] == 'crowdsource-verification']

outsourced_label_df.shape

(6440611, 4)

In [14]:
excluding_labels = {
    "Animated cartoon",
    "Animation",
    "Clay animation",
    "Clip art",
    "Comics",
    "Comic book",
    "Manga",
    "Illustration",
    "Fashion illustration",
    "Screenshot",
    "Web page",
    "Website",
    "Computer wallpaper",
    "Wallpaper",
    "Wallpaper paste",
    "Diagram",
    "Circuit diagram",
    "Chart",
    "Atlas",
    "Map",
    "Texture",
    "Pattern",
    "Paisley (Pattern)",
    "Still life",
    "Still life photography",
    "Computer program",
    "Antivirus software",
    "Educational software",
    "Enterprise software",
    "Graphics software",
    "Multimedia software",
    "Network software",
    "Office application software",
    "Software",
    "Software Developer",
    "Software engineering",
    "Tax software",
    "Utility software",
    "Video editing software",
    "Video game software"
}

# Filter out rows where DisplayName is in excluding_labels
clean_class_df = class_df[~class_df['DisplayName'].isin(excluding_labels)]

clean_class_df.shape

(20891, 2)

In [15]:
merged_label_df = inhouse_label_df.merge(clean_class_df, on='LabelName', how='inner')

print(merged_label_df.shape)
merged_label_df.head()

(14533532, 5)


,ImageID,Source,LabelName,Confidence,DisplayName
0,000002b66c9c498e,verification,/m/011xdj,1.0,Meringue
1,000002b66c9c498e,verification,/m/012mj,1.0,Alcoholic beverage
2,000002b66c9c498e,verification,/m/012yh1,1.0,Style
3,000002b66c9c498e,verification,/m/014sv8,1.0,Human eye
4,000002b66c9c498e,verification,/m/015c4z,1.0,Sitting


In [16]:
# Group by ImageID and aggregate DisplayName into a list
grouped_label_df = merged_label_df.groupby('ImageID')['DisplayName'].apply(list).reset_index(name='Labels')

print(grouped_label_df.shape)
grouped_label_df.head()


(5346494, 2)


,ImageID,Labels
0,000002b66c9c498e,"[Meringue, Alcoholic beverage, Style, Human ey..."
1,000002b97e5471a0,"[Style, Composite material, Shape, Infrastruct..."
2,000002c707c9895e,"[Style, Organ (Biology), Headgear, Cloak, Huma..."
3,0000048549557964,"[Advertising, Motor vehicle, Style, Travel, Co..."
4,000004f4400f6ec5,"[Style, T-shirt, Organ (Biology), Composite ma..."


In [17]:
final_df = clean_train_df[["ImageID", "Subset"]].merge(grouped_label_df, on="ImageID", how="inner")

print(final_df.shape)
final_df.head()

(4549125, 3)


,ImageID,Subset,Labels
0,4fa8054781a4c382,train,"[Organ (Biology), Adult, Chapel, Lighting, Var..."
1,d05c3e451f79174d,train,"[Advertising, Toy, Human eye, Beard, Smile, Pe..."
2,5bde9ad21e82deed,train,"[Lake district, Water resources, Waterway, Pan..."
3,b37f763ae67d0888,train,"[Social group, Grassland, Savanna, Vegetation,..."
4,3231e75bd7120b7a,train,"[Style, Black, Beauty, Black-and-white, Health..."


In [ ]:
import os

os.makedirs("data/Open_Images", exist_ok=True)
final_df.to_parquet("data/Open_Images/train_list.parquet", index=False)